# Initial preprocessing, integration, clustering, and dimensional reduction analyses

### Notebook setup

In [150]:
from itertools import chain
from itertools import product
import numpy as np
import anndata as ad
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import harmonypy as harmony
import seaborn as sns
from scipy import io
from scipy import sparse

# import scvelo as scv
# import cellrank as cr

In [151]:
import matplotlib
matplotlib.use('TkAgg')

In [152]:
import gc
gc.enable()

In [153]:
import os
os.chdir('/local/workdir/dwm269/scCardiacOrganoid/')

In [154]:
from scripts.py.utils import *
from scripts.py.plots import *

In [155]:
# from importlib import reload

# from scripts.py.scantils import *
# import scripts # get module reference for reload
# reload(scripts.py.scantils) # reload step 1
# from scripts.py.scantils import * # reload step 2

In [156]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(
    dpi_save=400, dpi=200, 
    transparent=False, 
    color_map="plasma"
)

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.5 scipy==1.9.3 pandas==1.5.2 scikit-learn==1.1.3 statsmodels==0.13.5 python-igraph==0.10.2 pynndescent==0.5.8


In [157]:
# Load metadata
meta = pd.read_csv("resources/metadata.csv")
meta = meta.loc[meta["source"]=="Hoang et al",]
meta = meta.loc[meta["include"],]

meta.index = list(range(0, len(meta)))
meta

,sample,include,data.dir,pattern,pattern_int,timepoint,time_int,cell_line,source,soupx,soupx_rho_GeneFull
0,D0_600um,True,/workdir/dwm269/scCardiacOrganoid/data/STARsol...,600um,600.0,D0,0,GCaMP6f hiPSCs,Hoang et al,False,NaN
1,D1_600um,True,/workdir/dwm269/scCardiacOrganoid/data/STARsol...,600um,600.0,D1,1,GCaMP6f hiPSCs,Hoang et al,False,NaN
2,D4_200um,True,/workdir/dwm269/scCardiacOrganoid/data/STARsol...,200um,200.0,D4,4,GCaMP6f hiPSCs,Hoang et al,True,0.076
3,D4_600um,True,/workdir/dwm269/scCardiacOrganoid/data/STARsol...,600um,600.0,D4,4,GCaMP6f hiPSCs,Hoang et al,True,0.070
4,D4_1000um,True,/workdir/dwm269/scCardiacOrganoid/data/STARsol...,1000um,1000.0,D4,4,GCaMP6f hiPSCs,Hoang et al,True,0.140
5,D6_600um,True,/workdir/dwm269/scCardiacOrganoid/data/STARsol...,600um,600.0,D6,6,GCaMP6f hiPSCs,Hoang et al,True,0.029
6,D8_600um,True,/workdir/dwm269/scCardiacOrganoid/data/STARsol...,600um,600.0,D8,8,GCaMP6f hiPSCs,Hoang et al,True,0.015
7,D12_600um,True,/workdir/dwm269/scCardiacOrganoid/data/STARsol...,600um,600.0,D12,12,GCaMP6f hiPSCs,Hoang et al,True,0.017
8,D21_200um_B,True,/workdir/dwm269/scCardiacOrganoid/data/STARsol...,200um,200.0,D21,21,GCaMP6f hiPSCs,Hoang et al,True,0.011
9,D21_600um,True,/workdir/dwm269/scCardiacOrganoid/data/STARsol...,600um,600.0,D21,21,GCaMP6f hiPSCs,Hoang et al,True,0.039


Utility functions that I use during analyses - can find them in `scripts/py/scantils.py`

## Load dataset(s)

Load in count mats & initialize adata object(s)

In [158]:
adata_list = []
for i in range(0,meta.shape[0]):
    print("Sample: " + meta["sample"][i])
    
    if meta["soupx"][i]:
        # Scrubbed matrices from soupx don't have features saved in 10x format, need manual loading
        tmp=sc.read_mtx(filename=meta["data.dir"][i]+"/Solo.out/GeneFull/soupx/matrix.mtx.gz").transpose()
        tmp.var_names = np.loadtxt(meta["data.dir"][i]+"/Solo.out/GeneFull/soupx/features.tsv.gz",dtype="str")
        tmp.obs_names = np.loadtxt(meta["data.dir"][i]+"/Solo.out/GeneFull/soupx/barcodes.tsv.gz",dtype="str")
        
        adata_list.append(tmp)    
    else:        
        adata_list.append(
            sc.read_10x_mtx(
                path=meta["data.dir"][i]+"/Solo.out/GeneFull/filtered", 
                var_names='gene_symbols',
                make_unique=True,
                cache=False 
            )
        )    
    adata_list[i].var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`
    
    adata_list[i].layers['counts'] = adata_list[i].X # save counts as a layer for future plotting
    
    print("     Loaded " + str(adata_list[i].shape[0]) + " cells and " + str(adata_list[i].shape[1]) + " genes...")

Sample: D0_600um
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/anndata/utils.py:111: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  warnings.warn(


     Loaded 10762 cells and 67049 genes...
Sample: D1_600um
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/anndata/utils.py:111: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  warnings.warn(


     Loaded 10993 cells and 67049 genes...
Sample: D4_200um
     Loaded 7921 cells and 67049 genes...
Sample: D4_600um
     Loaded 9038 cells and 67049 genes...
Sample: D4_1000um
     Loaded 11415 cells and 67049 genes...
Sample: D6_600um
     Loaded 7652 cells and 67049 genes...
Sample: D8_600um
     Loaded 10789 cells and 67049 genes...
Sample: D12_600um
     Loaded 7277 cells and 67049 genes...
Sample: D21_200um_B
     Loaded 5972 cells and 67049 genes...
Sample: D21_600um
     Loaded 15243 cells and 67049 genes...
Sample: D21_1000um_B
     Loaded 10534 cells and 67049 genes...


Add metadata to each anndata object

In [159]:
for i in range(0,meta.shape[0]): #samples
    for j in range(0,meta.shape[1]): #metadata features
        adata_list[i].obs[meta.columns[j]] = meta.iloc[i,j]

Draw knee plots to show quality of each dataset  
[code source](https://www.kallistobus.tools/tutorials/kb_building_atlas/python/kb_analysis_0_python/)

In [160]:
# import matplotlib.pyplot as plt
# expected_num_cells = 10000

# for i in range(0,meta.shape[0]):
#     knee = np.sort((np.array(adata_list[i].X.sum(axis=1))).flatten())[::-1]

#     fig, ax = plt.subplots(figsize=(10, 7))

#     ax.loglog(knee, range(len(knee)), linewidth=3, color="b")
#     #ax.axvline(x=knee[expected_num_cells], linewidth=3, color="k")
#     #ax.axhline(y=expected_num_cells, linewidth=3, color="k")

#     ax.set_xlabel("UMI Counts")
#     ax.set_ylabel("Set of Barcodes")
#     ax.set_title(adata_list[i].obs["sample"][0])

#     plt.xlim([0, 20000])
#     plt.grid(True, which="both")
#     plt.show()

## Filter and preprocess individual datasets

In [161]:
# gene/transcript, mito filters count filters
for i in range(0,meta.shape[0]):
    print(meta["sample"][i] + ': ' + str(adata_list[i].shape[0]) + " cells and " + str(adata_list[i].shape[1]) +' features...')  
    
    # Hard filters for feature and UMI counts
    sc.pp.filter_cells(
        adata_list[i],
        min_genes=500
    )
    sc.pp.filter_cells(
        adata_list[i], 
        min_counts=1500
    )
    
    # Hard filter for sparsely detected features
    sc.pp.filter_genes(
        adata_list[i],
        min_cells=5
    ) 
        
    adata_list[i].var['mito'] = adata_list[i].var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
    sc.pp.calculate_qc_metrics(
        adata_list[i], 
        qc_vars=['mito'], 
        percent_top=None, 
        log1p=False, 
        inplace=True
    )  
    
    # QC filter(s)
    adata_list[i] = adata_list[i][adata_list[i].obs.pct_counts_mito < 20, :] # less than 20% mito 
    
    print('After filtering: ' + str(adata_list[i].shape[0]) + " cells and " + str(adata_list[i].shape[1]) +' features...')  
    print(" ")


D0_600um: 10762 cells and 67049 features...
filtered out 135 cells that have less than 500 genes expressed
filtered out 546 cells that have less than 1500 counts
filtered out 37765 genes that are detected in less than 5 cells
After filtering: 9869 cells and 29284 features...
 
D1_600um: 10993 cells and 67049 features...
filtered out 21 cells that have less than 500 genes expressed
filtered out 23 cells that have less than 1500 counts
filtered out 37617 genes that are detected in less than 5 cells
After filtering: 10687 cells and 29432 features...
 
D4_200um: 7921 cells and 67049 features...
filtered out 116 cells that have less than 500 genes expressed
filtered out 191 cells that have less than 1500 counts
filtered out 35854 genes that are detected in less than 5 cells
After filtering: 7591 cells and 31195 features...
 
D4_600um: 9038 cells and 67049 features...
filtered out 13 cells that have less than 500 genes expressed
filtered out 83 cells that have less than 1500 counts
filtered 

In [162]:
for i in range(0,meta.shape[0]):
    sc.pl.violin(
        adata_list[i], 
        keys='total_counts'#['n_genes','n_counts']
    )

/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.ob

## Doublet removal via Scrublet

Estimate doublet scores w/ Scrublet

In [164]:
sc.settings.set_figure_params(
    fontsize=8
)

for i in range(0,meta.shape[0]):
    sce.pp.scrublet(
        adata_list[i]
    )

Running Scrublet
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:04)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
Embedding transcriptomes using PCA...
Automatically set threshold at doublet score = 0.63
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.3%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 3.3%
    Scrublet finished (0:01:03)
Running Scrublet
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:08)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
Embedding transcriptomes using PCA...
Automatically set threshold at doublet score = 0.64
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.3%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 3.2%
    Scrublet finished (0:01:05)
Running Scrublet
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:03)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


...

normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
Embedding transcriptomes using PCA...
Automatically set threshold at doublet score = 0.56
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 0.6%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 12.5%
    Scrublet finished (0:00:18)
Running Scrublet
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:02)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
Embedding transcriptomes using PCA...
Automatically set threshold at doublet score = 0.16
Detected doublet rate = 6.7%
Estimated detectable doublet fraction = 56.1%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 11.9%
    Scrublet finished (0:00:51)
Running Scrublet
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:06)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
Embedding transcriptomes using PCA...
Automatically set threshold at doublet score = 0.20
Detected doublet rate = 4.5%
Estimated detectable doublet fraction = 47.0%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 9.6%
    Scrublet finished (0:00:35)


In [165]:
for i in range(0,meta.shape[0]):
    sce.pl.scrublet_score_distribution(
        adata_list[i],
        figsize =[6,2.25]
    )

In [166]:
## Doublet score cutoff values for each sample
cutoff_threshold = [
    0.4, #D0_600um	
    0.4, #D1_600um	
    0.2, #D4_200um
    0.2, #D4_600um	
    0.2, #D4_1000um
    0.2, #D6_600um	
    0.2, #D8_600um
    0.2, #D12_600um
    0.2, #D20_600um
    0.2, # D21_200um_B
    0.2, #D21_600um
    0.2 #D21_1000um_B
]

## Remove doublets
print("Final cell & feature counts:\n")
for i in range(0,meta.shape[0]):    
    adata_list[i] = adata_list[i][adata_list[i].obs["doublet_score"] < cutoff_threshold[i],]
    print(meta["sample"][i] + ': ' + str(adata_list[i].shape[0]) + " cells and " + str(adata_list[i].shape[1]) +' features...')  
    print("")

Final cell & feature counts:

D0_600um: 9818 cells and 29284 features...

D1_600um: 10612 cells and 29432 features...

D4_200um: 7354 cells and 31195 features...

D4_600um: 8565 cells and 29680 features...

D4_1000um: 9966 cells and 30786 features...

D6_600um: 7160 cells and 30788 features...

D8_600um: 9686 cells and 31645 features...

D12_600um: 6036 cells and 29593 features...

D21_200um_B: 5247 cells and 27385 features...

D21_600um: 12938 cells and 30865 features...

D21_1000um_B: 9019 cells and 28159 features...



## Merged individual samples & preprocess merged object

Merge individual samples for integrated analysis

In [167]:
adata = adata_list[0].concatenate(
    adata_list[1:],
    index_unique=None
)

print(adata.shape)

/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


(96401, 24072)


In [168]:
adata.obs_names_make_unique()

In [169]:
sc.pp.normalize_total(
    adata, 
    target_sum=1e4
)

sc.pp.log1p(
    adata
)

normalizing counts per cell
    finished (0:00:03)


In [170]:
adata.layers['data'] = adata.X

In [171]:
sc.pp.highly_variable_genes(
    adata,
#     min_mean=0.0125, 
#     max_mean=3,
#     min_disp=0.5,
    subset=False
)

extracting highly variable genes
    finished (0:00:22)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [172]:
sc.settings.set_figure_params(
    fontsize=8,
    figsize=[4,4]
)
sc.pl.highly_variable_genes(adata)

In [173]:
sc.pp.scale(
    adata
#     max_value=10
)

sc.tl.pca(
    adata,
    svd_solver='arpack',
    n_comps=50
)

adata.obsm['pca'] = adata.obsm['X_pca'] # save PCA w/ out cell cycle regression

... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:01:27)


In [174]:
sc.settings.set_figure_params(
    fontsize=18,
    figsize=[8,7]
)

sc.pl.violin(
    adata,
    size=0.5,
    rotation=90,
    keys=['pct_counts_mito'],
    groupby='sample'
)

In [175]:
sc.pl.scatter(
    adata,
    size=8,
#     rotation=90,
    x='total_counts',
    y='n_genes',
    color='sample'
)

## Cell cycle regression & final preprocessing

Cell cycle scoring

In [176]:
# load in cell cycle genes lists
s_genes = list(pd.read_csv("resources/seurat_s_genes_2019.csv")['x'])
g2m_genes = list(pd.read_csv("resources/seurat_g2m_genes_2019.csv")['x'])

# score and plot
sc.tl.score_genes_cell_cycle(
    adata, 
    s_genes=s_genes, 
    g2m_genes=g2m_genes, 
    use_raw=False
)

calculating cell cycle phase
computing score 'S_score'


/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):


    finished: added
    'S_score', score of gene set (adata.obs).
    428 total control genes are used. (0:02:12)
computing score 'G2M_score'


/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/tools/_score_genes.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  for cut in np.unique(obs_cut.loc[gene_list]):


    finished: added
    'G2M_score', score of gene set (adata.obs).
    557 total control genes are used. (0:01:41)
-->     'phase', cell cycle phase (adata.obs)


In [177]:
sc.pp.regress_out(
    adata, 
    ['S_score', 'G2M_score']
)

regressing out ['S_score', 'G2M_score']
    finished (1:28:45)


In [178]:
sc.pp.scale(
    adata
#     max_value=10
)

sc.tl.pca(
    adata, 
    svd_solver='arpack',
    n_comps=50
)

adata.obsm['pca_cc'] = adata.obsm['X_pca']

computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:01:13)


In [179]:
sc.pl.violin(
    adata, 
    ['n_genes', 'total_counts', 'pct_counts_mito'], #'n_genes_by_counts',
    jitter=0.4, 
    multi_panel=True
)

In [180]:
sc.set_figure_params(
    figsize=(4,4),
    dpi=200,
    fontsize=10
)

sc.pl.scatter(adata, x='total_counts', y='pct_counts_mito')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')
sc.pl.highly_variable_genes(adata)

Finish preprocessing w/ PCA

## Harmony integration

- `harmony_sp` <- Perform batch correction on both sample ID and cell cycle phase; using PCA which does not include cell-cycle-regressed counts
- `harmony_s` <- batch correct just on sample ID, but using cell-cycle-regressed counts

In [181]:
# subset pca for less noisy Harmony integration? 
# adata.obsm['pca_sub'] = adata.obsm['pca'][:,1:30]

In [182]:
sce.pp.harmony_integrate(
    adata=adata, 
    key=["sample","phase"], 
    basis='pca',
    adjusted_basis='harmony_sp'
)

2022-12-13 11:45:28,164 - harmonypy - INFO - Iteration 1 of 10
2022-12-13 11:46:44,964 - harmonypy - INFO - Iteration 2 of 10
2022-12-13 11:48:09,479 - harmonypy - INFO - Iteration 3 of 10
2022-12-13 11:49:33,809 - harmonypy - INFO - Converged after 3 iterations


Re-order Harmony dimensions by decreasing % variance

In [183]:
reorder_reduction(
    adata,
    reduction="harmony_sp",
    verbose=True
)

Reduction variance by dimension:
[66.97038, 32.525, 38.14958, 23.312126, 9.015288, 8.621928, 16.705961, 11.50067, 13.608058, 5.350435, 6.508146, 6.1659803, 4.1095824, 3.3540812, 5.6777062, 4.8084044, 5.2410183, 4.0958495, 3.4207733, 3.921077, 3.7897596, 2.9329972, 2.8647203, 2.5667157, 2.4476993, 2.4602854, 2.5557456, 2.6350524, 3.3902571, 2.9314756, 2.3703935, 1.9783283, 2.3929849, 2.4321132, 2.1269596, 2.1551845, 2.1879969, 2.0231771, 2.3120604, 1.7690398, 2.380004, 1.6906623, 2.1003735, 2.076333, 2.0639749, 1.8825839, 1.918827, 1.9286175, 1.9080156, 1.7162254]


In [184]:
sce.pp.harmony_integrate(
    adata=adata, 
    key=["sample"], 
    basis="pca_cc",
    adjusted_basis="harmony_s"
)

2022-12-13 11:52:41,583 - harmonypy - INFO - Iteration 1 of 10
2022-12-13 11:54:05,985 - harmonypy - INFO - Iteration 2 of 10
2022-12-13 11:55:17,787 - harmonypy - INFO - Converged after 2 iterations


In [185]:
reorder_reduction(
    adata,
    reduction="harmony_s"
)

### Clustering

In [186]:
sc.pp.neighbors(
    adata,
    n_neighbors=50,
    n_pcs=npcs(adata,reduction="harmony_sp"),
    use_rep="harmony_sp",
    key_added="harmony_sp_neighbors"
)
gc.collect()

computing neighbors
    finished: added to `.uns['harmony_sp_neighbors']`
    `.obsp['harmony_sp_neighbors_distances']`, distances for each pair of neighbors
    `.obsp['harmony_sp_neighbors_connectivities']`, weighted adjacency matrix (0:01:01)


213

In [187]:
for res in [0.8, 1.0, 1.2]:
    sc.tl.leiden(
        adata,
        resolution=res, 
        neighbors_key="harmony_sp_neighbors",
        key_added=f"leiden_harmony_sp_{res}"
    )
    gc.collect()

running Leiden clustering
    finished: found 15 clusters and added
    'leiden_harmony_sp_0.8', the cluster labels (adata.obs, categorical) (0:02:14)
running Leiden clustering
    finished: found 17 clusters and added
    'leiden_harmony_sp_1.0', the cluster labels (adata.obs, categorical) (0:03:30)
running Leiden clustering
    finished: found 22 clusters and added
    'leiden_harmony_sp_1.2', the cluster labels (adata.obs, categorical) (0:03:07)


Repeat clustering on second harmony embedding

In [188]:
sc.pp.neighbors(
    adata,
    n_neighbors=50,
    n_pcs=npcs(adata,reduction="harmony_s"),
    use_rep="harmony_s",
    key_added="harmony_s_neighbors"
)
gc.collect()

computing neighbors
    finished: added to `.uns['harmony_s_neighbors']`
    `.obsp['harmony_s_neighbors_distances']`, distances for each pair of neighbors
    `.obsp['harmony_s_neighbors_connectivities']`, weighted adjacency matrix (0:00:58)


52

In [189]:
for res in [0.8, 1.0, 1.2]:
    sc.tl.leiden(
        adata,
        resolution=res, 
        neighbors_key="harmony_s_neighbors",
        key_added=f"leiden_harmony_s_{res}"
    )
    gc.collect()

running Leiden clustering
    finished: found 15 clusters and added
    'leiden_harmony_s_0.8', the cluster labels (adata.obs, categorical) (0:02:55)
running Leiden clustering
    finished: found 19 clusters and added
    'leiden_harmony_s_1.0', the cluster labels (adata.obs, categorical) (0:02:15)
running Leiden clustering
    finished: found 20 clusters and added
    'leiden_harmony_s_1.2', the cluster labels (adata.obs, categorical) (0:02:34)


## QC plots

In [190]:
sc.set_figure_params(
    # figsize=(14,16),
    # fontsize=26,
    dpi=300
)
sc.pl.violin(
    adata, 
    ['n_genes', 'total_counts', 'pct_counts_mito'],
    groupby="sample",
    jitter=0.4,
    rotation=90,
    size=0.01,
    multi_panel=True
)

In [191]:
adata.obs.groupby('pattern')['sample'].value_counts()

pattern  sample      
200um    D4_200um         7354
         D21_200um_B      5247
         D0_600um            0
         D1_600um            0
         D4_600um            0
         D4_1000um           0
         D6_600um            0
         D8_600um            0
         D12_600um           0
         D21_600um           0
         D21_1000um_B        0
600um    D21_600um       12938
         D1_600um        10612
         D0_600um         9818
         D8_600um         9686
         D4_600um         8565
         D6_600um         7160
         D12_600um        6036
         D4_200um            0
         D4_1000um           0
         D21_200um_B         0
         D21_1000um_B        0
1000um   D4_1000um        9966
         D21_1000um_B     9019
         D0_600um            0
         D1_600um            0
         D4_200um            0
         D4_600um            0
         D6_600um            0
         D8_600um            0
         D12_600um           0
         D21_200u

## PHATE

PHATE reduction to look at gradual changes in data 
[[source](https://nbviewer.jupyter.org/github/KrishnaswamyLab/PHATE/blob/master/Python/tutorial/EmbryoidBody.ipynb)]  
Great resource from PHATE authors on parameterization: [[link](https://dburkhardt.github.io/tutorial/visualizing_phate/#:~:text=PHATE%20has%20three%20key%20parameters,decay%20of%20the%20kernel%20tails)]

Compute PHATE embedding from the count matrix

In [192]:
sce.tl.phate(
    adata,
    # knn=100,
    # decay=40,
    t='auto',
    mds_solver='smacof',
    # n_pca=50,
    n_jobs=16
)

computing PHATE
Calculating PHATE...
  Running PHATE on 96401 observations and 24072 variables.
  Calculating graph and diffusion operator...
    Calculating PCA...


/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/phate/phate.py:186: FutureWarning: k is deprecated. Please use knn in future.
  warnings.warn("k is deprecated. Please use knn in future.", FutureWarning)
/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/phate/phate.py:190: FutureWarning: a is deprecated. Please use decay in future.
  warnings.warn("a is deprecated. Please use decay in future.", FutureWarning)


    Calculated PCA in 265.65 seconds.
    Calculating KNN search...
    Calculated KNN search in 683.22 seconds.
    Calculating affinities...
    Calculated affinities in 807.90 seconds.
  Calculated graph and diffusion operator in 1759.31 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 37.12 seconds.
    Calculating KMeans...
    Calculated KMeans in 32.08 seconds.
  Calculated landmark operator in 74.36 seconds.
  Calculating optimal t...
    Automatically selected t = 26
  Calculated optimal t in 8.36 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 3.34 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 11.08 seconds.
Calculated PHATE in 1856.48 seconds.
    finished: added
    'X_phate', PHATE coordinates (adata.obsm) (0:30:56)


In [202]:
sc.set_figure_params(
    figsize=(4,4),
    dpi=200
)
sc.pl.embedding(
    adata,
    basis='X_phate',
    color=["phase","time_int", "sample"],#['sample','leiden_harmony', 'phase', "time_int"],
#     edges=True, edges_width=0.005,
    palette='tab20b',
    legend_loc='on data',
#     legend_loc='right',
    legend_fontsize=8,
#      add_outline=True,
    size=3,
    cmap="plasma",
    sort_order=False,
    ncols=2
)

/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


Compute PHATE embedding from the Harmony-integrated reduced dimensions
- Using the number of harmony dimensions that contain >95% of variance

In [194]:
import phate

In [216]:
phate_operator = phate.PHATE(
    knn=10,
    decay=40,
    t='auto',
    mds_solver='smacof',
    n_jobs=42
)

# Use top N harmony dims which account for 95% of variance in harmony embedding
n_dims=npcs(adata,reduction="harmony_sp")
tmp_phate = phate_operator.fit_transform(adata.obsm["harmony_sp"][:,:n_dims]) 
adata.obsm['phate_harmony_sp'] = tmp_phate

Calculating PHATE...
  Running PHATE on 96401 observations and 41 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 477.30 seconds.
    Calculating affinities...
    Calculated affinities in 2.33 seconds.
  Calculated graph and diffusion operator in 480.30 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 28.04 seconds.
    Calculating KMeans...
    Calculated KMeans in 34.03 seconds.
  Calculated landmark operator in 66.56 seconds.
  Calculating optimal t...
    Automatically selected t = 25
  Calculated optimal t in 6.08 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 3.32 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 30.83 seconds.
Calculated PHATE in 587.46 seconds.


In [217]:
sc.set_figure_params(
    figsize=(4,4),
    dpi=200
)
sc.pl.embedding(
    adata,
    basis='phate_harmony_sp',
    color=['leiden_harmony_sp_1.0'],#
    # color=['sample','leiden_harmony_sp_10_types', 'phase', "time_int"],
#     edges=True, edges_width=0.005,
    palette='tab20b',
    legend_loc='on data',
#     legend_loc='right',
    legend_fontsize=8,
#      add_outline=True,
    size=1.5,
    sort_order=False,
    ncols=2
)

/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


In [197]:
sc.set_figure_params(
    figsize=(4,4),
    dpi=200
)
sc.pl.embedding(
    adata,
    basis='phate_harmony_sp',
    color=[ "doublet_score"], #'EPCAM','KDR', 'PECAM1',
    color_map='viridis',
    size=0.5,
    ncols=2,
    sort_order=True
)

Repeat PHATE with CC-regressed Harmony embedding

In [198]:
phate_operator = phate.PHATE(
    knn=5,
    decay=40,
    # gamma=1,
    # n_pca=20,
    t='auto',
    mds_solver='smacof',
    n_jobs=42
)

# Use top N harmony dims which account for 95% of variance in harmony embedding
n_dims=npcs(adata,reduction="harmony_s")
tmp_phate = phate_operator.fit_transform(adata.obsm["harmony_s"][:,:n_dims]) 
adata.obsm['phate_harmony_s'] = tmp_phate

Calculating PHATE...
  Running PHATE on 96401 observations and 42 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 466.28 seconds.
    Calculating affinities...
    Calculated affinities in 4.49 seconds.
  Calculated graph and diffusion operator in 471.46 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 22.85 seconds.
    Calculating KMeans...
    Calculated KMeans in 39.59 seconds.
  Calculated landmark operator in 67.77 seconds.
  Calculating optimal t...
    Automatically selected t = 31
  Calculated optimal t in 6.74 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 4.41 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 25.88 seconds.
Calculated PHATE in 576.31 seconds.


In [199]:
sc.set_figure_params(
    figsize=(4,4),
    dpi=200
)
sc.pl.embedding(
    adata,
    basis='phate_harmony_s',
    color=['sample','leiden_harmony_s_1.0', 'phase', "time_int"],
#     edges=True, edges_width=0.005,
    palette='tab20b',
    legend_loc='on data',
#     legend_loc='right',
    legend_fontsize=8,
#      add_outline=True,
    size=2,
    sort_order=False,
    ncols=2
)

/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/home/dwm269/miniconda3/envs/pyRNA6/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


In [200]:
sc.set_figure_params(
    figsize=(6,4),
    dpi=200
)
sc.pl.violin(
    adata,
    groupby='leiden_harmony_s_1.0',#['sample','leiden_harmony', 'phase', "time_int"],
    palette='tab20b',
    keys="doublet_score",
    size=0,
    inner="box"
)

### Compute/plot density across the PHATE embedding
##### https://scanpy.readthedocs.io/en/stable/generated/scanpy.tl.embedding_density.html

In [204]:
# sc.set_figure_params(figsize=(4,4),dpi=200)

# sc.tl.embedding_density(
#     adata,
#     basis='phate_harmony_sp', 
#     groupby='timepoint'
# )
# sc.pl.embedding_density(
#     adata,
#     basis='phate_harmony_sp',
#     key='phate_harmony_density_timepoint', 
#     bg_dotsize=5,
#     fg_dotsize=5,
#     ncols=3,
#     group=['D0','D1','D4','D6','D8','D21']
# )

## Save preprocessed object for future analyses

In [205]:
# Make sure that time points are saved as a continuous variable, not a categorical variable
adata.obs["time_int"] = adata.obs["time_int"].astype("float")

In [218]:
# Save h5ad object for subsequent python analyses
# adata.write("data/pyobjs/scCO_v7.h5ad")
adata.write("data/pyobjs/scCO_v1a.h5ad")

In [219]:
gc.collect()

40